In [1]:
import java.nio.file.Files
import java.nio.file.Path

data class BenchmarkData(val name: String, val score: Double, val error: Double) {
    override fun toString(): String = "$name: $score ± $error ns/op"
}

fun readHotspotBenchmarkData(benchmarkName: String) = Files.readAllLines(Path.of("build", "results", "jmh", "results.txt"))
    .mapNotNull {
        val r = Regex("$benchmarkName\\.([^ ]+) +avgt +\\d+ +([0-9.]+) +± +([0-9.]+) +ns/op")
        val match = r.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            match.groups[1]!!.value,
            match.groups[2]!!.value.toDouble(),
            match.groups[3]!!.value.toDouble(),
        )
    }
(readHotspotBenchmarkData("Wolf3dBenchmark") + readHotspotBenchmarkData("FibonacciBenchmark")).joinToString("\n")

baselineDouble: 3.4452692055E7 ± 179825.43 ns/op
baselineFloat: 3.3442081679E7 ± 155096.973 ns/op
inlineDouble: 3.5092982538E7 ± 629680.522 ns/op
inlineFloat: 3.3049489676E7 ± 132569.463 ns/op
longPackFloat: 5.6222059737E7 ± 187842.195 ns/op
mutableRefDouble: 4.0280109959E7 ± 160454.464 ns/op
mutableRefFloatIn1Long: 6.4718508885E7 ± 241321.275 ns/op
mutableRefFloatIn2Longs: 3.9739173887E7 ± 308392.329 ns/op
valueDouble: 2.4705608547E7 ± 136760.005 ns/op
valueFloat: 2.5182216781E7 ± 172483.878 ns/op
valueInlineDouble: 2.4759526397E7 ± 203118.971 ns/op
valueInlineFloat: 2.5109059249E7 ± 119419.088 ns/op
valuePreserveBoxDouble: 2.4760650008E7 ± 105223.719 ns/op
valuePreserveBoxFloat: 2.5129968165E7 ± 144684.654 ns/op
baselineInt: 383.797 ± 1.769 ns/op
baselineLong: 408.784 ± 6.732 ns/op
longPackInt: 192.441 ± 1.612 ns/op
mutableRefIntIn1Long: 303.433 ± 1.631 ns/op
mutableRefIntIn2Longs: 254.153 ± 1.15 ns/op
mutableRefLong: 244.623 ± 2.031 ns/op
valueInt: 367.184 ± 1.403 ns/op
valueLong: 3

In [2]:
%use lets-plot

In [3]:
import org.jetbrains.letsPlot.intern.PosKind
import org.jetbrains.letsPlot.intern.StatKind
import org.jetbrains.letsPlot.intern.layer.PosOptions
import org.jetbrains.letsPlot.intern.layer.StatOptions

fun makePlot(name: String, platform: String, benchmarkLines: List<BenchmarkData>) {
    if (benchmarkLines.isEmpty()) return
    val names = benchmarkLines.map { it.name }
    val scores = benchmarkLines.map { it.score }
    val scoresMinusErrors = benchmarkLines.map { it.score - it.error / 2 }
    val scoresPlusErrors = benchmarkLines.map { it.score + it.error / 2 }
    val baseline = listOf("Float", "Double", "Int", "Long").mapNotNull { type ->
        val result = benchmarkLines.singleOrNull { it.name == "baseline$type" }
        if (result != null) type to result else null
    }.toMap()
    val maxScore = scoresPlusErrors.max()
    val plot = letsPlot() +
            labs("$name ($platform)", fill = "Benchmark", y = "ns/op") +
            geomBar(stat = StatOptions(kind = StatKind.IDENTITY)) {
                x = names
                y = scores
                fill = names
            } +
            geomErrorBar(
                width = 0.5,
                position = PosOptions(kind = PosKind.DODGE),
                size = 0.6
            ) {
                x = names
                ymin = scoresMinusErrors
                ymax = scoresPlusErrors
            } +
            geomText(
                stat = StatOptions(kind = StatKind.IDENTITY),
                position = positionStack(vjust = 1.0),
                size = 5,
            ) {
                x = names
                y = scores.map {
                    val threshold = 0.08
                    if (it < maxScore * threshold) it + threshold * maxScore / 2 else it / 2
                }
                label = benchmarkLines.map {
                    val type = when {
                        it.name.contains("Float") -> "Float"
                        it.name.contains("Int") -> "Int"
                        it.name.contains("Double") -> "Double"
                        it.name.contains("Long") -> "Long"
                        else -> error("Unknown name: ${it.name}")
                    }
                    val score = it.score / baseline[type]!!.score
                    val error = it.error / baseline[type]!!.score
                    "${String.format("%.0f", score * 100)}%\n±\n${String.format("%.1f", error * 100)}%\n${type}"
                }
            } +
            scaleXDiscrete(
                labels = names,
                name = "",
                limits = names + listOf(""),
                breaks = names.toList()
            ) +
            scaleYContinuous(limits = 0 to maxScore, name = "ns/op") +
            theme(plotTitle = elementText(hjust = 0.5)).legendPositionRight() +
            ggsize(1000, 800)
    ggsave(plot = plot, filename = "$name ($platform).svg")
    plot.show()
}


In [4]:
makePlot("Wolf3d", "Hotspot", readHotspotBenchmarkData("Wolf3dBenchmark"))
makePlot("Fibonacci", "Hotspot", readHotspotBenchmarkData("FibonacciBenchmark"))

In [5]:
import java.io.FileFilter
import kotlin.io.path.listDirectoryEntries

fun readArtBenchmarkData(benchmarkName: String) = Path.of("android-benchmark/build/outputs/androidTest-results/connected")
    .toFile().listFiles(FileFilter { it.isDirectory })!!.single()
    .listFiles { dir, name -> name.startsWith("logcat-org.jetbrains.") }!!.flatMap { it.readLines() }
    .mapNotNull {
        val pattern = Regex("^\\d{2}-\\d{2} +\\d+:\\d+:\\d+.\\d+ +\\d+ +\\d+ +I +Benchmark: +$benchmarkName\\.([^\\[]+)\\[Metric \\(timeNs\\) +results: +median +([^,]+), +min [^,]+, +max [^,]+, +standardDeviation: +([^,]+), +.*")
        val match = pattern.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            match.groups[1]!!.value,
            match.groups[2]!!.value.toDouble(),
            match.groups[3]!!.value.toDouble(),
        )
     }.let { lines ->
         val indexes = readHotspotBenchmarkData(benchmarkName).mapIndexed { index, benchmarkData -> benchmarkData.name to index }.toMap()
         if (lines.all { it.name in indexes }) lines.sortedBy { indexes[it.name]!! } else lines
     }
(readArtBenchmarkData("Wolf3dBenchmark") + readArtBenchmarkData("FibonacciBenchmark")).joinToString("\n")

baselineDouble: 4.109314225E8 ± 3.785773000388188E7 ns/op
baselineFloat: 3.422955765E8 ± 2.2952346541190278E7 ns/op
inlineDouble: 8.5637973E8 ± 5.4032954877539895E7 ns/op
inlineFloat: 6.632091725E8 ± 1.3761962879633907E7 ns/op
longPackFloat: 1.30840923E8 ± 2536042.6577650066 ns/op
mutableRefDouble: 1.06645961E8 ± 1626549.6038457053 ns/op
mutableRefFloatIn1Long: 1.59625634E8 ± 2380701.772825691 ns/op
mutableRefFloatIn2Longs: 1.246060385E8 ± 549912.3055080409 ns/op
valueDouble: 4.63633923E8 ± 1.735101079385085E7 ns/op
valueFloat: 3.58129654E8 ± 6399666.819567365 ns/op
valueInlineDouble: 1.86353808E7 ± 203653.63720301932 ns/op
valueInlineFloat: 2.3094235625E7 ± 707453.0228103918 ns/op
valuePreserveBoxDouble: 4.31563019E8 ± 1.0779540313346304E7 ns/op
valuePreserveBoxFloat: 2.852673265E8 ± 9893948.625326464 ns/op
baselineInt: 1523.277832867574 ± 68.64011825711752 ns/op
baselineLong: 2058.3171853555396 ± 114.82394964796192 ns/op
longPackInt: 175.15255678639363 ± 6.721273577379677 ns/op
mutab

In [6]:
makePlot("Wolf3d", "ART", readArtBenchmarkData("Wolf3dBenchmark"))
makePlot("Fibonacci", "ART", readArtBenchmarkData("FibonacciBenchmark"))